In [22]:

import pandas as pd
from difflib import SequenceMatcher
import os

In [3]:
df = pd.read_csv('totalset.csv', compression = 'gzip')
pd.set_option('display.max_columns', None)
print(df.head())

                   identifier  \
0  ddd:010728187:mpeg21:a0005   
1  ddd:010728188:mpeg21:a0001   
2  ddd:010728188:mpeg21:a0002   
3  ddd:010728188:mpeg21:a0003   
4  ddd:010731495:mpeg21:a0001   

                                             gt text  \
0  NEDERLANDEN. Valencijn, den 23 July. Sedert de...   
1  VRANCKRYCK. PArijs, den 12 Augusti. Den Hertog...   
2  DUYTSLANDT, &amp;c. Straetsburg, den 12 Aug. G...   
3  NEDERLANDEN. Uyt het Leger onder sijne Hooghey...   
4  Ordinarise Middel-weeckse Courante, No. 30. 16...   

                                            ocr text    CER    WER  \
0  NEDERLANDEN. Vakncijn, den j -> July. Scdeit d...  34.66  49.34   
1  VRANKRYK. PÜAriif , den 12 Den Hertog van Sund...  11.86  37.43   
2  DUYTSLANDT, & C. Strertil-H,?, den ft Gifteren...  31.87  70.23   
3  NEDERLANDEN. iVl.t het. Leger tndtr fi'yne Hoo...  17.50  51.32   
4  Uyt Madrid , den 30 Jury. Stère .ïliajeflept t...  49.03  84.17   

   WER (order independent)  dictionary look

In [4]:
test= df[df['identifier'] =='ddd:010728187:mpeg21:a0005']
test

,identifier,gt text,ocr text,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,source,word count gt,word count ocr
0,ddd:010728187:mpeg21:a0005,"NEDERLANDEN. Valencijn, den 23 July. Sedert de...","NEDERLANDEN. Vakncijn, den j -> July. Scdeit d...",34.66,49.34,46.1,87.99,51.68,0.254021,0.22,Meertens,1968,1791


In [5]:
# split GT and OCR-output of each row into sentences (based on full stops).
for index, row in test.iterrows():
    gt_sentences = row['gt text'].split(".")
    ocr_sentences = row['ocr text'].split(".")

In [6]:
match_list = []

for sentence in gt_sentences:
    for match in ocr_sentences:
        if SequenceMatcher(None, sentence, match).ratio() > 0.75:
            match_list.extend([[sentence, match]])

In [7]:
match_list

[['NEDERLANDEN', 'NEDERLANDEN'],
 [' Valencijn, den 23 July', ' Vakncijn, den j -> July'],
 [' Charleroy, den 24 July', ' Chariêrty, den 24 Juiy'],
 [' Monsr', ' Monlr'],
 [' Brussel, den 27 July', ' Bruffél, den z 7 July'],
 [" Den 21 deser zijn de Franse voor Arien gekomen, onder den Maerschalck d\\'Humieres, ende hadden d\\'onse gedwongen het Fort St",
  " Den 11 defer zijn de Fi anfevoor Arien geicomen , onder den i\\ aerfchalck d'Humicrcs, ende hadden d'onfegedwor"],
 [' Lauarens te verlaten Binnen de voorsz',
  ' Laurens te vei laten Binnen de vooriz'],
 [" Stad is een seer out Soldaet tot Gouverneur gemaeckt, genaemt Comte de Broné, en is maer een advenue om aen te konnen komen, en buyten dat altemael Moeras, soo dat lang sal konnen houden; Ondertusschen is den Hertogh van Villa Hermosa met 16000 man in marsch, als mede den Velt-Maerschalck Waldeck, om de selve plaets t\\'ontsetten",
  " Stad is een feer out Soldaet tot Gouverneur gemaeckt genaemt Comte de brooé, en is macr een 

In [8]:
matched_gt_sentences = []
matched_ocr_sentences = []

for sentence in match_list:
    if len(sentence[0]) or len(sentence[1]) > 0:
        matched_gt_sentences.append(sentence[0])
        matched_ocr_sentences.append(sentence[1])

In [9]:
data = {'GT': matched_gt_sentences, 'OCR': matched_ocr_sentences }
matched = pd.DataFrame(data=data)

In [10]:
matched

,GT,OCR
0,NEDERLANDEN,NEDERLANDEN
1,"Valencijn, den 23 July","Vakncijn, den j -> July"
2,"Charleroy, den 24 July","Chariêrty, den 24 Juiy"
3,Monsr,Monlr
4,"Brussel, den 27 July","Bruffél, den z 7 July"
5,Den 21 deser zijn de Franse voor Arien gekome...,Den 11 defer zijn de Fi anfevoor Arien geicom...
6,Lauarens te verlaten Binnen de voorsz,Laurens te vei laten Binnen de vooriz
7,Stad is een seer out Soldaet tot Gouverneur g...,Stad is een feer out Soldaet tot Gouverneur g...
8,Het resterende Franse Leger is van Kivrain op...,Htt refterende Franfe Leger is van Kivrain op...
9,Schans,Schans


In [11]:
# .join() with lists
separator = '.'
doc_GT = separator.join(matched_gt_sentences)
doc_OCR = separator.join(matched_ocr_sentences)

"NEDERLANDEN. Vakncijn, den j -> July. Chariêrty, den 24 Juiy. Monlr. Bruffél, den z 7 July. Den 11 defer zijn de Fi anfevoor Arien geicomen , onder den i\\ aerfchalck d'Humicrcs, ende hadden d'onfegedwor. Laurens te vei laten Binnen de vooriz. Stad is een feer out Soldaet tot Gouverneur gemaeckt genaemt Comte de brooé, en is macr een auvenue om om aen te konnen komen, en buyten dat altemael MoerasJ, foo dat lang fal konnen houden; Ondertuffchen is den Hcrtogh van Villa liermofamet ióooomanin marfch , als mede den Velt-Maerfchalck Waldeck, om de felve t'ontfettcn. Htt refterende Franfe Leger is van Kivrain opgebroocken, ende heeft veileden nacht tot Pirrues gelegen , ontrent een uur van Condé op de wegh v. Schans. Van dcfen nacht fal den Capiceyn vande Mineurs bet mineren beginnen, om te zien of men de-Mijne kan ontdecken. Gift er en begon men op her Baftion la Reyne te canonneren, dat geloovc in een dag of twee zijn bekomft fal om fangen. Defen namiddag zijn alhier aengekomen twee Sch

In [17]:
#print(f"{test['identifier'].values[0]}.txt")

ddd:010728187:mpeg21:a0005.txt


In [31]:
#f = open(f"{test['identifier'].values[0]}.txt","w+")
print(type(doc_OCR))

<class 'str'>


In [32]:
save_path = r"C:\Users\Gebruiker\Desktop\Thesis\Testbestanden\Evaluatietest"
filename = "idtest_test_OCR.txt"
file_ocr = open(os.path.join(save_path, filename),"w+", encoding="utf-8")
file_ocr.write(doc_OCR)
file_ocr.close()

In [34]:
filename = "idtest_test_GT.txt"
file_gt = open(os.path.join(save_path, filename),"w+", encoding="utf-8")
file_gt.write(doc_GT)
file_gt.close()